In [6]:
import requests as r
from Bio import SeqIO
from io import StringIO
import pandas as pd
from HelperFunctions import find_the_index_of_the_modification
import re

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [9]:
jaydes_data = pd.read_csv('../datasets/combined_modified_peptide.tsv', sep = '\t')
jaydes_data = jaydes_data.dropna(subset=['Assigned Modifications'])

In [10]:
jaydes_data

,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,Protein ID,Entry Name,Gene,Protein Description,Mapped Genes,Mapped Proteins,1 Spectral Count,10 Spectral Count,11 Spectral Count,12 Spectral Count,2 Spectral Count,3 Spectral Count,4 Spectral Count,5 Spectral Count,6 Spectral Count,7 Spectral Count,8 Spectral Count,9 Spectral Count,1 Intensity,10 Intensity,11 Intensity,12 Intensity,2 Intensity,3 Intensity,4 Intensity,5 Intensity,6 Intensity,7 Intensity,8 Intensity,9 Intensity,1 Match Type,10 Match Type,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type
6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,K,S,576,587,12,3,10M(655.3735),sp|Q8C196|CPSM_MOUSE,Q8C196,CPSM_MOUSE,Cps1,"Carbamoyl-phosphate synthase [ammonia], mitoch...",NaN,NaN,0,0,2,0,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,2.543694e+07,0.000000e+00,0.000000e+00,0.000000e+00,3.002489e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched
9,AAEMLLFGK,AAEM[649.3660]LLFGK,K,K,293,301,9,3,4M(649.3660),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,0,0,0,0,0,1,0,0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.472334e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched
10,AAEMLLFGK,AAEM[655.3735]LLFGK,K,K,293,301,9,3,4M(655.3735),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,3,2,1,1,2,2,1,3,2,0,3,2,1.788667e+07,1.369351e+07,1.423012e+07,1.586156e+07,1.548381e+07,1.539164e+07,1.527121e+07,1.608834e+07,1.515103e+07,0.000000e+00,2.254254e+07,1.556738e+07,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,unmatched,MS/MS,MS/MS
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,K,K,81,92,12,3,7M(649.3660),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,3,3,3,3,3,3,3,3,2,3,3,3,1.050958e+08,1.086895e+08,1.075148e+08,9.866490e+07,1.228555e+08,1.036483e+08,1.062934e+08,1.112873e+08,1.068229e+08,1.029083e+08,1.148145e+08,1.193007e+08,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,K,K,81,92,12,3,7M(655.3735),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,2,2,2,1,2,2,2,2,2,2,1,1,1.083520e+08,1.144818e+08,1.116604e+08,1.005727e+08,1.231404e+08,1.084914e+08,1.042410e+08,1.134865e+08,1.104685e+08,1.024969e+08,1.145532e+08,1.243082e+08,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5447,STFRLALIQLQVSSIKSDNLTR,n[42.0106]STFRLALIQLQVSSIKSDNLTR,M,A,2,23,22,3,N-term(42.0106),sp|Q9JHW2|NIT2_MOUSE,Q9JHW2,NIT2_MOUSE,Nit2,Omega-amidase NIT2,NaN,NaN,0,1,1,0,0,2,0,0,1,0,1,1,0.000000e+00,4.166723e+07,3.457027e+07,0.000000e+00,0.000000e+00,3.973615e+07,0.000000e+00,0.000000e+00,3.961702e+07,0.000000e+00,4.001150e+07,3.330879e+07,unmatched,MS/MS,MS/MS,unmatched,unmatched,MS/MS,unmatched,unmatched,MS/MS,unmatched,MS/MS,MS/MS
5448,TDAAVSFAK,n[42.0106]TDAAVSFAK,M,D,2,10,9,2,N-term(42.0106),sp|P51881|ADT2_MOUSE,P51881,ADT2_MOUSE,Slc25a5,ADP/ATP translocase 2,NaN,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.088644e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched
5449,TDAAVSFAKDFLAGGVA

In [19]:
proteins_of_interest = jaydes_data[jaydes_data['Assigned Modifications'].str.contains('M(655.3735)', regex=False) | jaydes_data['Assigned Modifications'].str.contains('M(649.3660)', regex=False)]









In [20]:
proteins_of_interest

,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,Protein ID,Entry Name,Gene,Protein Description,Mapped Genes,Mapped Proteins,1 Spectral Count,10 Spectral Count,11 Spectral Count,12 Spectral Count,2 Spectral Count,3 Spectral Count,4 Spectral Count,5 Spectral Count,6 Spectral Count,7 Spectral Count,8 Spectral Count,9 Spectral Count,1 Intensity,10 Intensity,11 Intensity,12 Intensity,2 Intensity,3 Intensity,4 Intensity,5 Intensity,6 Intensity,7 Intensity,8 Intensity,9 Intensity,1 Match Type,10 Match Type,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type
6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,K,S,576,587,12,3,10M(655.3735),sp|Q8C196|CPSM_MOUSE,Q8C196,CPSM_MOUSE,Cps1,"Carbamoyl-phosphate synthase [ammonia], mitoch...",NaN,NaN,0,0,2,0,0,0,1,0,0,0,0,0,0.0,0.0,25436938.0,0.0,0.0,0.0,30024890.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched
9,AAEMLLFGK,AAEM[649.3660]LLFGK,K,K,293,301,9,3,4M(649.3660),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,8472334.0,0.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched
10,AAEMLLFGK,AAEM[655.3735]LLFGK,K,K,293,301,9,3,4M(655.3735),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,3,2,1,1,2,2,1,3,2,0,3,2,17886672.0,13693514.0,14230118.0,15861561.0,15483814.0,15391638.0,15271206.0,16088336.0,15151029.0,0.0,22542544.0,15567377.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,unmatched,MS/MS,MS/MS
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,K,K,81,92,12,3,7M(649.3660),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,3,3,3,3,3,3,3,3,2,3,3,3,105095824.0,108689472.0,107514768.0,98664904.0,122855464.0,103648304.0,106293424.0,111287272.0,106822864.0,102908304.0,114814464.0,119300728.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,K,K,81,92,12,3,7M(655.3735),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,2,2,2,1,2,2,2,2,2,2,1,1,108352024.0,114481752.0,111660384.0,100572704.0,123140424.0,108491376.0,104240984.0,113486536.0,110468520.0,102496872.0,114553192.0,124308184.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,-,F,1,12,12,4,"N-term(42.0106),2M(655.3735)",sp|Q9CQ54|NDUC2_MOUSE,Q9CQ54,NDUC2_MOUSE,Ndufc2,NADH dehydrogenase [ubiquinone] 1 subunit C2,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21944120.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,M,E,2,16,15,4,"N-term(42.0106),11M(649.3660)",sp|P24270|CATA_MOUSE,P24270,CATA_MOUSE,Cat,Catalase,NaN,NaN,0,1,0,0,1,1,0,1,0,1,0,1,0.0,23581108.0,0.0,0.0,22891132.0,23278660.0,0.0,3849613.0,0.0,22662232.0,0.0,23301186.0,unmatched,MS/MS,unmatched,unmatched,MS/MS,MS/MS,unmatched,MS/MS,unmatched,MS/MS,unmatched,MS/MS
5442,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[655.3735]KQWK,M,E,2,16,15,4,"N-term(42.0106),11M(655.3735)",sp|P24270|CATA_MOUSE,P24270,CATA_MOUSE,Cat,Catalase,NaN,NaN,1,0,1,1,0,0,0,0,0,0,1,1,24666958.0,0.0,23210620.0,23845792.0,0.0,0.0,0.0,0.0,0.0,0.0,25350372.0,3595287.0,MS/MS,unmatched,MS/MS,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,MS/MS
5445,SSMTQNLR,n[

In [39]:
def get_protein_seq(cID):
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))

    return str(pSeq[0].seq)


In [42]:
protein_sequence = get_protein_seq('Q9WUR2')
print(protein_sequence[292:301])

AAEMLLFGK


In [44]:
proteins_of_interest_with_completed_seq = proteins_of_interest.copy()
proteins_of_interest_with_completed_seq['Complete Sequence'] = proteins_of_interest['Protein ID'].apply(get_protein_seq)

In [45]:
proteins_of_interest_with_completed_seq

,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,Protein ID,Entry Name,Gene,Protein Description,Mapped Genes,Mapped Proteins,1 Spectral Count,10 Spectral Count,11 Spectral Count,12 Spectral Count,2 Spectral Count,3 Spectral Count,4 Spectral Count,5 Spectral Count,6 Spectral Count,7 Spectral Count,8 Spectral Count,9 Spectral Count,1 Intensity,10 Intensity,11 Intensity,12 Intensity,2 Intensity,3 Intensity,4 Intensity,5 Intensity,6 Intensity,7 Intensity,8 Intensity,9 Intensity,1 Match Type,10 Match Type,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type,Complete Sequence
6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,K,S,576,587,12,3,10M(655.3735),sp|Q8C196|CPSM_MOUSE,Q8C196,CPSM_MOUSE,Cps1,"Carbamoyl-phosphate synthase [ammonia], mitoch...",NaN,NaN,0,0,2,0,0,0,1,0,0,0,0,0,0.0,0.0,25436938.0,0.0,0.0,0.0,30024890.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,MTRILTACKVVKTLKSGFGFANVTTKRQWDFSRPGIRLLSVKAKTA...
9,AAEMLLFGK,AAEM[649.3660]LLFGK,K,K,293,301,9,3,4M(649.3660),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,8472334.0,0.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
10,AAEMLLFGK,AAEM[655.3735]LLFGK,K,K,293,301,9,3,4M(655.3735),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,3,2,1,1,2,2,1,3,2,0,3,2,17886672.0,13693514.0,14230118.0,15861561.0,15483814.0,15391638.0,15271206.0,16088336.0,15151029.0,0.0,22542544.0,15567377.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,unmatched,MS/MS,MS/MS,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,K,K,81,92,12,3,7M(649.3660),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,3,3,3,3,3,3,3,3,2,3,3,3,105095824.0,108689472.0,107514768.0,98664904.0,122855464.0,103648304.0,106293424.0,111287272.0,106822864.0,102908304.0,114814464.0,119300728.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,K,K,81,92,12,3,7M(655.3735),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,2,2,2,1,2,2,2,2,2,2,1,1,108352024.0,114481752.0,111660384.0,100572704.0,123140424.0,108491376.0,104240984.0,113486536.0,110468520.0,102496872.0,114553192.0,124308184.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,-,F,1,12,12,4,"N-term(42.0106),2M(655.3735)",sp|Q9CQ54|NDUC2_MOUSE,Q9CQ54,NDUC2_MOUSE,Ndufc2,NADH dehydrogenase [ubiquinone] 1 subunit C2,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21944120.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,MMNGRPGHEPLKFLPDEARSLPPPKLNDPRLVYMGLLGYCTGLMDN...
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,M,E,2,16,15,4,"N-term(42.0106),11M(649.3660)",sp|P24270|CATA_MOUSE,P24270,CATA_MOUSE,Cat,Catalase,NaN,NaN,0,1,0,0,1,1,0,1,0,1,0,1,0.0,23581108.0,0.0,0.0,22891132.0,23278660.0,0.0,3849613.0,0.0,22662232.0,0.0,23301186.0,unmatched,MS/MS,unmatched,unmatched,MS/MS,MS/MS,unmatched,MS/MS,unmatched,MS/MS,unmatched,MS/MS,MSDSRDPASDQMKQWKEQRASQRPDVLTTGGGNPIGDKLNIMTAGS...
5442,SDSRDPAS

In [46]:
proteins_of_interest_with_completed_seq.to_csv('proteins_of_interest_with_completed_seq.csv')

In [49]:
proteins_of_interest_with_completed_seq

,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,Protein ID,Entry Name,Gene,Protein Description,Mapped Genes,Mapped Proteins,1 Spectral Count,10 Spectral Count,11 Spectral Count,12 Spectral Count,2 Spectral Count,3 Spectral Count,4 Spectral Count,5 Spectral Count,6 Spectral Count,7 Spectral Count,8 Spectral Count,9 Spectral Count,1 Intensity,10 Intensity,11 Intensity,12 Intensity,2 Intensity,3 Intensity,4 Intensity,5 Intensity,6 Intensity,7 Intensity,8 Intensity,9 Intensity,1 Match Type,10 Match Type,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type,Complete Sequence
6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,K,S,576,587,12,3,10M(655.3735),sp|Q8C196|CPSM_MOUSE,Q8C196,CPSM_MOUSE,Cps1,"Carbamoyl-phosphate synthase [ammonia], mitoch...",NaN,NaN,0,0,2,0,0,0,1,0,0,0,0,0,0.0,0.0,25436938.0,0.0,0.0,0.0,30024890.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,MTRILTACKVVKTLKSGFGFANVTTKRQWDFSRPGIRLLSVKAKTA...
9,AAEMLLFGK,AAEM[649.3660]LLFGK,K,K,293,301,9,3,4M(649.3660),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,8472334.0,0.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
10,AAEMLLFGK,AAEM[655.3735]LLFGK,K,K,293,301,9,3,4M(655.3735),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,3,2,1,1,2,2,1,3,2,0,3,2,17886672.0,13693514.0,14230118.0,15861561.0,15483814.0,15391638.0,15271206.0,16088336.0,15151029.0,0.0,22542544.0,15567377.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,unmatched,MS/MS,MS/MS,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,K,K,81,92,12,3,7M(649.3660),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,3,3,3,3,3,3,3,3,2,3,3,3,105095824.0,108689472.0,107514768.0,98664904.0,122855464.0,103648304.0,106293424.0,111287272.0,106822864.0,102908304.0,114814464.0,119300728.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,K,K,81,92,12,3,7M(655.3735),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,2,2,2,1,2,2,2,2,2,2,1,1,108352024.0,114481752.0,111660384.0,100572704.0,123140424.0,108491376.0,104240984.0,113486536.0,110468520.0,102496872.0,114553192.0,124308184.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,-,F,1,12,12,4,"N-term(42.0106),2M(655.3735)",sp|Q9CQ54|NDUC2_MOUSE,Q9CQ54,NDUC2_MOUSE,Ndufc2,NADH dehydrogenase [ubiquinone] 1 subunit C2,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21944120.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,MMNGRPGHEPLKFLPDEARSLPPPKLNDPRLVYMGLLGYCTGLMDN...
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,M,E,2,16,15,4,"N-term(42.0106),11M(649.3660)",sp|P24270|CATA_MOUSE,P24270,CATA_MOUSE,Cat,Catalase,NaN,NaN,0,1,0,0,1,1,0,1,0,1,0,1,0.0,23581108.0,0.0,0.0,22891132.0,23278660.0,0.0,3849613.0,0.0,22662232.0,0.0,23301186.0,unmatched,MS/MS,unmatched,unmatched,MS/MS,MS/MS,unmatched,MS/MS,unmatched,MS/MS,unmatched,MS/MS,MSDSRDPASDQMKQWKEQRASQRPDVLTTGGGNPIGDKLNIMTAGS...
5442,SDSRDPAS

In [52]:
proteins_of_interest_with_completed_seq = proteins_of_interest_with_completed_seq.reset_index()
proteins_of_interest_with_completed_seq 

,level_0,index,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,Protein ID,Entry Name,Gene,Protein Description,Mapped Genes,Mapped Proteins,1 Spectral Count,10 Spectral Count,11 Spectral Count,12 Spectral Count,2 Spectral Count,3 Spectral Count,4 Spectral Count,5 Spectral Count,6 Spectral Count,7 Spectral Count,8 Spectral Count,9 Spectral Count,1 Intensity,10 Intensity,11 Intensity,12 Intensity,2 Intensity,3 Intensity,4 Intensity,5 Intensity,6 Intensity,7 Intensity,8 Intensity,9 Intensity,1 Match Type,10 Match Type,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type,Complete Sequence
0,0,6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,K,S,576,587,12,3,10M(655.3735),sp|Q8C196|CPSM_MOUSE,Q8C196,CPSM_MOUSE,Cps1,"Carbamoyl-phosphate synthase [ammonia], mitoch...",NaN,NaN,0,0,2,0,0,0,1,0,0,0,0,0,0.0,0.0,25436938.0,0.0,0.0,0.0,30024890.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,MTRILTACKVVKTLKSGFGFANVTTKRQWDFSRPGIRLLSVKAKTA...
1,1,9,AAEMLLFGK,AAEM[649.3660]LLFGK,K,K,293,301,9,3,4M(649.3660),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,8472334.0,0.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
2,2,10,AAEMLLFGK,AAEM[655.3735]LLFGK,K,K,293,301,9,3,4M(655.3735),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,3,2,1,1,2,2,1,3,2,0,3,2,17886672.0,13693514.0,14230118.0,15861561.0,15483814.0,15391638.0,15271206.0,16088336.0,15151029.0,0.0,22542544.0,15567377.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,unmatched,MS/MS,MS/MS,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
3,3,11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,K,K,81,92,12,3,7M(649.3660),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,3,3,3,3,3,3,3,3,2,3,3,3,105095824.0,108689472.0,107514768.0,98664904.0,122855464.0,103648304.0,106293424.0,111287272.0,106822864.0,102908304.0,114814464.0,119300728.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
4,4,12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,K,K,81,92,12,3,7M(655.3735),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,2,2,2,1,2,2,2,2,2,2,1,1,108352024.0,114481752.0,111660384.0,100572704.0,123140424.0,108491376.0,104240984.0,113486536.0,110468520.0,102496872.0,114553192.0,124308184.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,729,5431,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,-,F,1,12,12,4,"N-term(42.0106),2M(655.3735)",sp|Q9CQ54|NDUC2_MOUSE,Q9CQ54,NDUC2_MOUSE,Ndufc2,NADH dehydrogenase [ubiquinone] 1 subunit C2,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21944120.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,MMNGRPGHEPLKFLPDEARSLPPPKLNDPRLVYMGLLGYCTGLMDN...
730,730,5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,M,E,2,16,15,4,"N-term(42.0106),11M(649.3660)",sp|P24270|CATA_MOUSE,P24270,CATA_MOUSE,Cat,Catalase,NaN,NaN,0,1,0,0,1,1,0,1,0,1,0,1,0.0,23581108.0,0.0,0.0,22891132.0,23278660.0,0.0,3849613.0,0.0,22662232.0,0.0,23301186.0,unmatched,MS/MS,unmatched,unmatched,MS/MS,MS/MS,unmatched,MS/MS,unmatched,MS/MS,unmatched,MS/MS,MSDSR

In [53]:
proteins_of_interest_with_completed_seq_reset_index = proteins_of_interest_with_completed_seq.drop(['index', 'level_0'], axis=1)
proteins_of_interest_with_completed_seq_reset_index

,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,Protein ID,Entry Name,Gene,Protein Description,Mapped Genes,Mapped Proteins,1 Spectral Count,10 Spectral Count,11 Spectral Count,12 Spectral Count,2 Spectral Count,3 Spectral Count,4 Spectral Count,5 Spectral Count,6 Spectral Count,7 Spectral Count,8 Spectral Count,9 Spectral Count,1 Intensity,10 Intensity,11 Intensity,12 Intensity,2 Intensity,3 Intensity,4 Intensity,5 Intensity,6 Intensity,7 Intensity,8 Intensity,9 Intensity,1 Match Type,10 Match Type,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type,Complete Sequence
0,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,K,S,576,587,12,3,10M(655.3735),sp|Q8C196|CPSM_MOUSE,Q8C196,CPSM_MOUSE,Cps1,"Carbamoyl-phosphate synthase [ammonia], mitoch...",NaN,NaN,0,0,2,0,0,0,1,0,0,0,0,0,0.0,0.0,25436938.0,0.0,0.0,0.0,30024890.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,MTRILTACKVVKTLKSGFGFANVTTKRQWDFSRPGIRLLSVKAKTA...
1,AAEMLLFGK,AAEM[649.3660]LLFGK,K,K,293,301,9,3,4M(649.3660),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,8472334.0,0.0,0.0,0.0,0.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
2,AAEMLLFGK,AAEM[655.3735]LLFGK,K,K,293,301,9,3,4M(655.3735),sp|Q9WUR2|ECI2_MOUSE,Q9WUR2,ECI2_MOUSE,Eci2,Enoyl-CoA delta isomerase 2,Eci3,sp|Q78JN3|ECI3_MOUSE,3,2,1,1,2,2,1,3,2,0,3,2,17886672.0,13693514.0,14230118.0,15861561.0,15483814.0,15391638.0,15271206.0,16088336.0,15151029.0,0.0,22542544.0,15567377.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,unmatched,MS/MS,MS/MS,MAAVTWSRARCWCPSVLQVFRLQVAKLHLGRPTMRASQQDFENALN...
3,AAESSAMAATEK,AAESSAM[649.3660]AATEK,K,K,81,92,12,3,7M(649.3660),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,3,3,3,3,3,3,3,3,2,3,3,3,105095824.0,108689472.0,107514768.0,98664904.0,122855464.0,103648304.0,106293424.0,111287272.0,106822864.0,102908304.0,114814464.0,119300728.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
4,AAESSAMAATEK,AAESSAM[655.3735]AATEK,K,K,81,92,12,3,7M(655.3735),sp|Q9Z1P6|NDUA7_MOUSE,Q9Z1P6,NDUA7_MOUSE,Ndufa7,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,NaN,NaN,2,2,2,1,2,2,2,2,2,2,1,1,108352024.0,114481752.0,111660384.0,100572704.0,123140424.0,108491376.0,104240984.0,113486536.0,110468520.0,102496872.0,114553192.0,124308184.0,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MASATRVIQKLRNWASGQDLQAKLQLRYQEIAKRTQPPPKLPVGPS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,-,F,1,12,12,4,"N-term(42.0106),2M(655.3735)",sp|Q9CQ54|NDUC2_MOUSE,Q9CQ54,NDUC2_MOUSE,Ndufc2,NADH dehydrogenase [ubiquinone] 1 subunit C2,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21944120.0,0.0,0.0,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,MMNGRPGHEPLKFLPDEARSLPPPKLNDPRLVYMGLLGYCTGLMDN...
730,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,M,E,2,16,15,4,"N-term(42.0106),11M(649.3660)",sp|P24270|CATA_MOUSE,P24270,CATA_MOUSE,Cat,Catalase,NaN,NaN,0,1,0,0,1,1,0,1,0,1,0,1,0.0,23581108.0,0.0,0.0,22891132.0,23278660.0,0.0,3849613.0,0.0,22662232.0,0.0,23301186.0,unmatched,MS/MS,unmatched,unmatched,MS/MS,MS/MS,unmatched,MS/MS,unmatched,MS/MS,unmatched,MS/MS,MSDSRDPASDQMKQWKEQRASQRPDVLTTGGGNPIGDKLNIMTAGS...
731,SDSRDPASDQMKQW

# checking to make sure we extracted the correct protein sequences


In [57]:
for i in range(proteins_of_interest_with_completed_seq_reset_index.shape[0]):
    seq1 = proteins_of_interest_with_completed_seq_reset_index['Complete Sequence'][i]
    seq2 = proteins_of_interest_with_completed_seq_reset_index['Peptide Sequence'][i]
    start_pos = proteins_of_interest_with_completed_seq_reset_index['Start'][i]
    end_pos = proteins_of_interest_with_completed_seq_reset_index['End'][i]
    if (seq1[start_pos - 1: end_pos] != seq2):
        print(f'mismatch at index {i}, original sequence:{seq2}, fetched sequence{seq1}')
    

In [20]:
def find_the_index_of_the_modification(modified_peptide_seq, amino_acid, complete_protein_sequence):
    # Extract the modification and its index in the modified peptide sequence
    modifications = re.findall(fr'{amino_acid}\[.*?\]', modified_peptide_seq)
    if (not modifications):
        return -1
    number_of_modifications = len(modifications)

    return number_of_modifications
        
    

    # mod_index = modified_peptide_seq.index('[' + modification + ']')

    # # Find the index of the amino acid in the complete protein sequence
    # amino_acid_index = complete_protein_sequence.find(amino_acid)

    # # Find the offset caused by any modifications before the amino acid
    # offset = mod_index - modified_peptide_seq.index(amino_acid)

    # # Calculate the index in the complete protein sequence
    # index_in_protein = amino_acid_index + offset

    # return index_in_protein  

In [21]:
peptide = "AAM[4666]TIGYPVM[649.3660]IR"
find_the_index_of_the_modification(peptide, 'M', "s")

['M[4666]', 'M[649.3660]']
